In [70]:
def A(n,k):
    ret = 1
    for i in range(k):
        ret *= (n - i)
    return ret
def C(n,k):
    return A(n,k) // A(k,k)

def pre_C(N):
    c = [[0 for _ in range(N+1)] for _ in range(N+1)]
    c[0][0] = 1
    for i in range(1,N+1):
        c[i][0] = 1
        c[i][i] = 1
        for j in range(1,i):
            c[i][j] = c[i-1][j] + c[i-1][j-1]
    return c

def weighted_sum(arr, debug = False):
    N = len(arr) - 1
    W = 2.0 ** N
    c = pre_C(N)
    ret = 0
    for i in range(N+1):
        p = c[N][i] / W
        if debug:
            print("{0:04}: {1:.7f}, {2:.7f}, {3}".format(i, p * arr[i], p, arr[i]))
        ret += p * arr[i]
    return ret

pre_C(10)

[[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 2, 1, 0, 0, 0, 0, 0, 0, 0, 0],
 [1, 3, 3, 1, 0, 0, 0, 0, 0, 0, 0],
 [1, 4, 6, 4, 1, 0, 0, 0, 0, 0, 0],
 [1, 5, 10, 10, 5, 1, 0, 0, 0, 0, 0],
 [1, 6, 15, 20, 15, 6, 1, 0, 0, 0, 0],
 [1, 7, 21, 35, 35, 21, 7, 1, 0, 0, 0],
 [1, 8, 28, 56, 70, 56, 28, 8, 1, 0, 0],
 [1, 9, 36, 84, 126, 126, 84, 36, 9, 1, 0],
 [1, 10, 45, 120, 210, 252, 210, 120, 45, 10, 1]]

In [8]:
def compute_current(N):
    dp = [[0 for _ in range(N+1)] for _ in range(N+1)]
    for k in range(N):
        dp[k][k+1] = 2
        for n in range(k+2, N+1):
            dp[k][n] = 2 * ((n-1 - k) / (n-1) * dp[k][n-1] + k / (n-1))
    return dp

compute_current(5)

[[0, 2, 4.0, 8.0, 16.0, 32.0],
 [0, 0, 2, 3.0, 4.666666666666667, 7.5],
 [0, 0, 0, 2, 2.6666666666666665, 3.6666666666666665],
 [0, 0, 0, 0, 2, 2.5],
 [0, 0, 0, 0, 0, 2],
 [0, 0, 0, 0, 0, 0]]

In [18]:
def compute_winned(N):
    cur_dp = compute_current(N)
    dp = [[0 for _ in range(N+1)] for _ in range(N+1)]
    for n in range(1,N+1):
        for k in range(n+1):
            if k > 0:
                dp[n][k] += (k / n) * (dp[n-1][k-1] + cur_dp[k-1][n])
            dp[n][k] += ((n - k) / n) * dp[n-1][k]
    return dp, cur_dp

compute_winned(2)
            

([[0, 0, 0], [0.0, 2.0, 0], [0.0, 3.0, 4.0]],
 [[0, 2, 4.0], [0, 0, 2], [0, 0, 0]])

In [45]:
def check_win_dp(N):
    c = pre_C(N)
    win_dp, cur_dp = compute_winned(N)
    W = 2**N
    ret = 0.0
    for i in range(N+1):
        p = c[N][i] / W
        #print(p, win_dp[N][i], p * win_dp[N][i])
        ret += p * win_dp[N][i]
    print(ret, (N - 1) * (N + 4) / 4 + 1)
        
    cur_W = 2**(N-1)
    cur = 0.0
    for i in range(N):
        p = c[N-1][i] / cur_W
        cur += p * cur_dp[i][N]
    print(cur, N + 1)

check_win_dp(100)


2574.9999999999995 2575.0
100.99999999999999 101


In [500]:
def lose_last_game(N):
    c = pre_C(N)
    win_dp, cur_dp = compute_winned(N)
    dp = [0 for _ in range(N+1)]
    for k in range(1,N+1):
        dp[k] = win_dp[N-1][k-1] + cur_dp[k-1][N]
    return dp

ret = lose_last_game(100)
#print(ret)
print(weighted_sum(ret[1:], True))

def percent_min_winned(N, P):
    dp = lose_last_game(N)
    dp = dp[1:]
    n = len(dp) - 1
    W = 2 ** n
    c = pre_C(N)
    ret_winned = 0.0
    ret_p = 0.0
    for i in range(n,-1,-1):
        p = c[n][i] / W
        if ret_p + p >= P:
            return (ret_p, dp[i+1]), (ret_p+p, dp[i])
        ret_p += p
percent_min_winned(100, 0.5)

0000: 2.0000000, 0.0000000, 1.2676506002282294e+30
0001: 4.0000000, 0.0000000, 2.5609103034913723e+28
0002: 6.0000000, 0.0000000, 7.839521337218488e+26
0003: 8.0000000, 0.0000000, 3.2327923040076232e+25
0004: 10.0000000, 0.0000000, 1.683745991670637e+24
0005: 12.0000000, 0.0000000, 1.063418521055139e+23
0006: 14.0000000, 0.0000000, 7.919074092963802e+21
0007: 16.0000000, 0.0000000, 6.812106746635528e+20
0008: 18.0000000, 0.0000000, 6.664017469534755e+19
0009: 20.0000000, 0.0000000, 7.323096120367864e+18
0010: 22.0000000, 0.0000000, 8.950450813782945e+17
0011: 24.0000000, 0.0000000, 1.2068023569145546e+17
0012: 26.0000000, 0.0000000, 1.782776209078319e+16
0013: 28.0000000, 0.0000000, 2868835278976602.0
0014: 30.0000000, 0.0000000, 500378246333123.1
0015: 32.0000000, 0.0000000, 94188846368582.25
0016: 34.0000000, 0.0000000, 19062028431730.406
0017: 36.0000000, 0.0000000, 4133933876753.4385
0018: 38.0000000, 0.0000000, 957862727532.0895
0019: 40.0000000, 0.0000000, 236509315430.63937
0020

((0.4204107626128213, 571.7773105248815), (0.5, 640.7901579945999))

In [587]:
import random

def once():
    b = 1
    while(random.SystemRandom().randint(0,1)) :
        b += 1
    return b

def run(n, c):
    i = 0
    e = 0
    while(i < n):
        e -= c
        b = once()
        e += (1 << b)
        i += b
    return e

def sample(m, n, c):
    e = 0
    for i in range(m):
        e += run(n, c)
    return e / m

sample(1, 100, 0)

354.0